In [1]:
import numpy as np #linear algebra
import pandas as pd #data processing 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

# Importing into dataframe
# df_Title_Basics = pd.read_csv("./Prj_Data/DownLoadedData_Imdb/title.basics (1).tsv.gz", delimiter='\t', encoding='ANSI')
df_Name_Basics = pd.read_csv("./Prj_Data/DownLoadedData_Imdb/name.basics (1).tsv.gz", delimiter='\t', encoding='ANSI')

# df_Generes_With_tconst = pd.read_excel("df_Generes_With_tconst.xlsx")
df_IMDB_MovieCatalog = pd.read_excel("df_IMDB_MovieCatalog.xlsx")
df_starsplayers_knownForMovie = pd.read_excel ('./Prj_Data/ImdbScrapingData/TheNumbers/output/df_starsplayers_knownForMovies.xlsx', sheet_name='Export')
df_starsplayers = pd.read_excel ('./Prj_Data/ImdbScrapingData/TheNumbers/input/PeopleMetaData.xlsx', sheet_name='Export')
df_Principles = pd.read_csv("./Prj_Data/DownLoadedData_Imdb/title.principals (1).tsv.gz", delimiter='\t', encoding='ANSI')


In [2]:
#_________________________________Inserting nconst and tconst into Numbers Meatadata actor tables________________________

In [3]:
#1. Start by Narrowing the Basics and Principles tables down by only getting data that can be linked to our 
#main movies tconts/ tabls
df_IMDB_MovieCatalog_Short = df_IMDB_MovieCatalog[["tconst","titleyear"]]

In [4]:
ActorswithMoviesWeCareAbout = df_IMDB_MovieCatalog_Short.merge(df_Principles, on="tconst", how="inner").merge(df_Name_Basics, on="nconst",how="inner")


In [5]:
#2. Now that you have reduced the number of possible linking rows for our Star Players Table, link on name_IMBD
#  Remember this first requires you to make sure the names in IMDB (Robert Downey, JR vs Robert Downey J.R link up
# via updating the People metadata table manually cross referencing via eye and cut and paste)


In [6]:
#get star actors connected to nconst
df_starsplayers_WithMixedNconst = df_starsplayers.merge(ActorswithMoviesWeCareAbout,left_on="Name_IMDB", right_on="primaryName", how="outer", indicator="left")
df_starsplayers_WithMixedNconst = df_starsplayers_WithMixedNconst.drop_duplicates()


In [7]:
#------SAVE----Used to create cross reference for names in IMDB to names in Numbers (see to facilitate the above step)
# df_starsplayers_MixedNconst = df_starsplayers.merge(ActorswithMoviesWeCareAbout,left_on="Name_IMDB", right_on="primaryName", how="outer", indicator="outer")
# df_starsplayers_MixedNconst = df_starsplayers_MixedNconst.drop_duplicates()
# #get star actors connected to nconst

In [8]:
# Make sure not missing any important people when inserting nconst
# df_starsplayers_MixedNconst_Null_nconst = df_starsplayers_MixedNconst[df_starsplayers_MixedNconst['nconst'].isnull()]
# df_starsplayers_MixedNconst[df_starsplayers_MixedNconst['nconst'].isnull()].sort_values(by="Name")

In [9]:
#DF 1 NCONST ONLY
df_StarActors_With_nconst = df_starsplayers_WithMixedNconst[['Rank','Name_IMDB', 'Name', 'Domestic Box Office', 'Movies', 'Average', 'Link', 'Contribution', 'BirthdayClean', 'nameyear', 'nconst']]
df_StarActors_With_nconst = df_StarActors_With_nconst.drop_duplicates()
df_StarActors_With_nconst = df_StarActors_With_nconst.dropna(subset=["Name"])

In [10]:
df_StarActors_With_nconst.head()

,Rank,Name_IMDB,Name,Domestic Box Office,Movies,Average,Link,Contribution,BirthdayClean,nameyear,nconst
0,407.0,Aaron Eckhart,Aaron Eckhart,483257969.0,21.0,23012284.0,https://www.the-numbers.com/person/43480401-Aaron-Eckhart,Actor,"March 12, 1968",Aaron Eckhart1968,nm0001173
21,495.0,Aaron Ryder,Aaron Ryder,261059123.0,16.0,16316195.0,https://www.the-numbers.com/person/233220401-Aaron-Ryder,Producer,"February 22, 1975",Aaron Ryder1975,nm0753083
32,161.0,Aaron Sorkin,Aaron Sorkin,533178310.0,10.0,53317831.0,https://www.the-numbers.com/person/178570401-Aaron-Sorkin,Writer,"June 9, 1961",Aaron Sorkin1961,nm0815070
38,237.0,Aaron Taylor-Johnson,Aaron Taylor-Johnson,810600479.0,10.0,81060048.0,https://www.the-numbers.com/person/790600401-Aaron-Taylor-Johnson,Actor,"June 13, 1990",Aaron Taylor-Johnson1990,nm1093951
50,1462.0,Aaron Yoo,Aaron Yoo,81159365.0,1.0,81159365.0,https://www.the-numbers.com/person/156090401-Aaron-Yoo,Actor,"June 21, 1982",Aaron Yoo1982,nm1683094


In [11]:
df_StarActors_With_nconst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2629 entries, 0 to 20026
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Rank                 2629 non-null   float64
 1   Name_IMDB            2629 non-null   object 
 2   Name                 2629 non-null   object 
 3   Domestic Box Office  2629 non-null   float64
 4   Movies               2629 non-null   float64
 5   Average              2629 non-null   float64
 6   Link                 2629 non-null   object 
 7   Contribution         2629 non-null   object 
 8   BirthdayClean        2629 non-null   object 
 9   nameyear             2629 non-null   object 
 10  nconst               2460 non-null   object 
dtypes: float64(4), object(7)
memory usage: 246.5+ KB


In [12]:
df_StarActors_With_nconst["A_Player"] = 0
df_StarActors_With_nconst["A_Player"] = df_StarActors_With_nconst.apply(lambda row: 1 if ((row["Contribution"] =="Actor") and (row["Movies"]>=7) and (row["Average"]>=40000000)) else row["A_Player"], axis=1)
df_StarActors_With_nconst["A_Player"] = df_StarActors_With_nconst.apply(lambda row: 1 if ((row["Contribution"] =="Director") and (row["Movies"]>=11) and (row["Average"]>=100000000)) else row["A_Player"], axis=1)
df_StarActors_With_nconst["A_Player"] = df_StarActors_With_nconst.apply(lambda row: 1 if ((row["Contribution"] =="Writer") and (row["Movies"]>=5) and (row["Average"]>=100000000)) else row["A_Player"], axis=1)
df_StarActors_With_nconst["A_Player"] = df_StarActors_With_nconst.apply(lambda row: 1 if ((row["Contribution"] =="Producer") and (row["Movies"]>=9) and (row["Average"]>=100000000)) else row["A_Player"], axis=1)


# and (row["Average"]>=100000000)
# and (row["Movie"]>=7)

In [13]:
df_StarActors_With_nconst.to_excel("df_StarActors_With_nconst.xlsx")

In [14]:
#DF 2 NCONST AND TCONST
df_StarActors_With_nconst_tconst = df_starsplayers_WithMixedNconst[['Rank', 'Name_IMDB', 'Name', 'Domestic Box Office', 'Movies', 'Average',
       'Link', 'Contribution', 'BirthdayClean', 'nameyear', 'nconst', 'tconst']]
df_StarActors_With_nconst_tconst = df_StarActors_With_nconst_tconst.drop_duplicates()
df_StarActors_With_nconst_tconst = df_StarActors_With_nconst_tconst.dropna(subset=["Name"])
df_StarActors_With_nconst_tconst["A_Player"] = 0
df_StarActors_With_nconst_tconst["A_Player"] = df_StarActors_With_nconst_tconst.apply(lambda row: 1 if ((row["Contribution"] =="Actor") and (row["Movies"]>=7) and (row["Average"]>=40000000)) else row["A_Player"], axis=1)
df_StarActors_With_nconst_tconst["A_Player"] = df_StarActors_With_nconst_tconst.apply(lambda row: 1 if ((row["Contribution"] =="Director") and (row["Movies"]>=11) and (row["Average"]>=100000000)) else row["A_Player"], axis=1)
df_StarActors_With_nconst_tconst["A_Player"] = df_StarActors_With_nconst_tconst.apply(lambda row: 1 if ((row["Contribution"] =="Writer") and (row["Movies"]>=5) and (row["Average"]>=100000000)) else row["A_Player"], axis=1)
df_StarActors_With_nconst_tconst["A_Player"] = df_StarActors_With_nconst_tconst.apply(lambda row: 1 if ((row["Contribution"] =="Producer") and (row["Movies"]>=9) and (row["Average"]>=100000000)) else row["A_Player"], axis=1)


In [15]:
df_StarActors_With_nconst_tconst.to_excel("df_StarActors_With_nconst_tconst.xlsx")

In [16]:
#________________________CODE FOR CHECKING THE ABOVE FOR ERRORS,ETC

In [17]:
df_StarActors_With_nconst[["Name","Contribution","nconst"]].sort_values(by="Name").groupby(["Name", "Contribution"]).count()

,,nconst
Name,Contribution,
Aaron Eckhart,Actor,1
Aaron Ryder,Producer,1
Aaron Sorkin,Writer,1
Aaron Taylor-Johnson,Actor,1
Aaron Yoo,Actor,1
Abbey Lee,Actor,1
Abby Kohn,Writer,1
Abigail Breslin,Actor,1
Adam Beach,Actor,1


In [18]:
# df_StarActors_With_nconst_tconst[["Name","Contribution","nconst","tconst"]].sort_values(by="Name").groupby(["Name", "Contribution"]).count()

df_StarActors_With_nconst_tconst[["Name","Contribution","tconst", "Movies"]].sort_values(by="Name").groupby(["Name", "Contribution"]).agg({'tconst':'count','Movies':'max'})


,,tconst,Movies
Name,Contribution,,
Aaron Eckhart,Actor,21,21.0
Aaron Ryder,Producer,11,16.0
Aaron Sorkin,Writer,6,10.0
Aaron Taylor-Johnson,Actor,12,10.0
Aaron Yoo,Actor,4,1.0
Abbey Lee,Actor,3,4.0
Abby Kohn,Writer,5,6.0
Abigail Breslin,Actor,16,14.0
Adam Beach,Actor,7,3.0


In [19]:
#_________________________END________________________________________________________________________________________

In [20]:
#_________________________insert tconstinto KnownforMoviesTable_________________________________________

In [21]:
df_starsplayers_knownForMovie["IMDB_XREF"] = df_starsplayers_knownForMovie["IMDB_XREF"].fillna("0")
df_starsplayers_knownForMovie["ReleaseYear"] = df_starsplayers_knownForMovie["ReleaseYear"].astype(str)

df_starsplayers_knownForMovie["ReleaseYear"] = df_starsplayers_knownForMovie["ReleaseYear"].apply(lambda x: str(x)[:-2])

df_starsplayers_knownForMovie["title_linktoIMDB"] = df_starsplayers_knownForMovie["IMDB_XREF"].str.title()

df_starsplayers_knownForMovie["titleyear_titlelinktoIMDB"] = df_starsplayers_knownForMovie["title_linktoIMDB"] + df_starsplayers_knownForMovie["ReleaseYear"]

In [22]:
df_IMDB_MovieCatalog["primaryTitle"] = df_IMDB_MovieCatalog["primaryTitle"].str.title()
df_IMDB_MovieCatalog["titleyear1"] = df_IMDB_MovieCatalog["primaryTitle"] + df_IMDB_MovieCatalog["startYear_str"].astype(str)

# df_IMDB_MovieCatalog["titleyear"] = df_IMDB_MovieCatalog.apply(lambda row : row["primaryTitle"] + str(row["startYear_str"]))

In [23]:
df_IMDB_MovieCatalog_Short = df_IMDB_MovieCatalog[[ "primaryTitle","titleyear1","tconst"]]

In [24]:
df_KnownForMerged = df_starsplayers_knownForMovie.merge(df_IMDB_MovieCatalog_Short, left_on="titleyear_titlelinktoIMDB" , right_on="titleyear1", how="left")

In [25]:
df_KnownForMerged.to_excel("df_StarActors_KnownForWithTconst.xlsx")

In [26]:
#_______________________________USED TO UPDATE TITLE MANUALLY in Star Play DF Titltes KNOWN FOR(500) DAYS OF SUMMER vs 500 Days of Summer 
df_KnownForMerged_nulls = df_KnownForMerged[["title_linktoIMDB","titleyear1","tconst"]][df_KnownForMerged["tconst"].isnull()]
df_KnownForMerged_nulls = df_KnownForMerged_nulls.drop_duplicates()

In [27]:
# code for checking for errors
df_IMDB_MovieCatalog[["titleyear1","tconst"]][df_IMDB_MovieCatalog["titleyear"].str.contains('12 Years A Slave')]

,titleyear1,tconst
100,12 Years A Slave2013,tt2024544


In [28]:
#___________________________________END________________________

In [ ]:
#________________________Remove Dups FROM Star Players Table... just asssume/Uses Primary Contributions_________________________________________

In [ ]:
df_Stars_no_dups = df_starsplayers.sort_values(by=["Name", "Contribution"])
df_Stars_no_dups1 = df_Stars_no_dups.drop_duplicates(subset="Name", keep="first")

In [ ]:
df_Stars_no_dups1.to_excel("df_StarActors_NoDups.xlsx")